# Examples
Fetching urls concurrently, and processing the responses as completed, is used as an example.  This simple task is nonetheless suprisingly tedious, especially using `asyncio`.

In [ ]:
# only needed for notebook
import conf
import nest_asyncio
nest_asyncio.apply()  

urls = [f'http://httpbin.org/delay/{d}' for d in (0.2, 0.1, 0.0)]

## Threaded

In [ ]:
from concurrent import futures
import requests

def fetch_all(urls):
    with requests.Session() as session, futures.ThreadPoolExecutor() as executor:
        fs = [executor.submit(session.get, url) for url in urls]
        for future in futures.as_completed(fs):
            yield future.result()

for resp in fetch_all(urls):
    print(resp.url)

`futured.threaded` abstracts away the boilerplate.

In [ ]:
from futured import threaded

fetch = threaded(requests.Session().get)
for resp in fetch.map(urls, as_completed=True):
    print(resp.url)

## Asynced

In [ ]:
import asyncio
import aiohttp

async def fetch_all(urls):
    async with aiohttp.ClientSession() as session:
        for future in asyncio.as_completed(map(session.get, urls)):
            yield await future

for resp in fetch_all(urls):
    print(resp.url)

The problem is coroutines support the `yield` keyword, but only to create async iterators.  Even though `asyncio.as_completed` is itself a normal iterator, there is _no way_ to write this generator as intended.  Additionally there is no iterator equivalent of `loop.run_until_complete`, to mitigate the viral nature of the `async` keyword.

So `futured.asynced` provides one.

In [ ]:
from futured import asynced

for resp in asynced.run(fetch_all, urls):
    print(resp.url)

The alternative approach is to explicitly handle the loop in the implementation.

In [ ]:
def fetch_all(urls):
    loop = asyncio.get_event_loop()
    session = aiohttp.ClientSession()
    for future in asyncio.as_completed(map(session.get, urls)):
        yield loop.run_until_complete(future)

for resp in fetch_all(urls):
    print(resp.url)

That works, although `aiohttp` is opinionated about only using `ClientSession` in an `async with` block.

`asynced` provides the same abstraction as `threaded`.

In [ ]:
fetch = asynced(aiohttp.ClientSession().get)
for resp in fetch.map(urls, as_completed=True):
    print(resp.url)